In [2]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)
import os
os.chdir('..')

要运行下面的例子，你可能需要安装以下依赖包：
```shell
pip install python-dotenv
pip install langchain-experimental
pip install pandas
pip install tabulate
pip install langchain-openai
```

# 在langchain中智能体

## create_pandas_dataframe_agent

### 导入智能体

In [4]:
# 今天的主角 pandas 智能体
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

### 准备数据集

In [5]:
# 反正是投喂给智能体，你可以随便准备一份 pandas 数据
import pandas as pd
df = pd.read_csv("data/titanic.csv")

### 先试试 OpenAI 的效果

In [10]:
from langchain_openai import ChatOpenAI
llm_openai = ChatOpenAI(model_name="gpt-4-1106-preview")
agent_executor = create_pandas_dataframe_agent(
        llm_openai,
        df,
        agent_type="openai-tools",
        verbose=True
    )
agent_executor.invoke("船上一共有多少人")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'len(df)'}`


891船上一共有 891 人。

> Finished chain.


{'input': '船上一共有多少人', 'output': '船上一共有 891 人。'}

### 再试试 ChatZhipuAI 效果

**注意：** 智能体应用对智商有要求，必须 `glm-4` 以上；`glm-3-turbo` 无法完成推理。

In [12]:
from langchain_zhipu import ChatZhipuAI
llm_zhipuai = ChatZhipuAI(model="glm-4")
agent_executor = create_pandas_dataframe_agent(
        llm_zhipuai,
        df,
        agent_type="openai-tools", # ZhipuAI的工具结构与OpenAI兼容，直接借用了
        verbose=True
    )
agent_executor.invoke("船上一共有多少人")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'len(df)'}`
responded: 我们可以通过查看 `df` 的行数来得知船上一共有多少人。在 Python 中，可以使用 `len()` 函数得到 DataFrame 的行数。这里使用 `python_repl_ast` 工具来执行相应的命令。<|assistant|>

891船上一共有891人。

> Finished chain.


{'input': '船上一共有多少人', 'output': '船上一共有891人。'}

In [20]:
from langchain_zhipu import ChatZhipuAI
from langchain.agents import AgentType
llm_zhipuai = ChatZhipuAI(model="glm-4")
agent_executor = create_pandas_dataframe_agent(
        llm_zhipuai,
        df,
        agent_type="openai-tools",
        verbose=True
    )
agent_executor.invoke("这个数据集中包含哪些信息？")



> Entering new AgentExecutor chain...
这个数据集看起来是关于泰坦尼克号乘客的信息。根据 `df.head()` 的输出，数据集中包含以下列：

1. **PassengerId**：乘客的唯一标识符。
2. **Survived**：一个指示乘客是否幸存的标志，1表示幸存，0表示未幸存。
3. **Pclass**：乘客的舱位等级，1代表一等舱，2代表二等舱，3代表三等舱。
4. **Name**：乘客的全名。
5. **Sex**：乘客的性别。
6. **Age**：乘客的年龄。
7. **SibSp**：乘客在船上的兄弟姐妹和配偶的数量。
8. **Parch**：乘客在船上的父母和孩子的数量。
9. **Ticket**：乘客的票号。
10. **Fare**：乘客支付的票价。
11. **Cabin**：乘客的客舱号码，如果数据不存在则是NaN。
12. **Embarked**：乘客上船的港口，S代表南安普敦，C代表 Cherbourg，Q代表 Queenstown。

这些信息可以用来分析乘客的生存率与各种因素（如性别、舱位等级、年龄等）之间的关系。

> Finished chain.


{'input': '这个数据集中包含哪些信息？',
 'output': '这个数据集看起来是关于泰坦尼克号乘客的信息。根据 `df.head()` 的输出，数据集中包含以下列：\n\n1. **PassengerId**：乘客的唯一标识符。\n2. **Survived**：一个指示乘客是否幸存的标志，1表示幸存，0表示未幸存。\n3. **Pclass**：乘客的舱位等级，1代表一等舱，2代表二等舱，3代表三等舱。\n4. **Name**：乘客的全名。\n5. **Sex**：乘客的性别。\n6. **Age**：乘客的年龄。\n7. **SibSp**：乘客在船上的兄弟姐妹和配偶的数量。\n8. **Parch**：乘客在船上的父母和孩子的数量。\n9. **Ticket**：乘客的票号。\n10. **Fare**：乘客支付的票价。\n11. **Cabin**：乘客的客舱号码，如果数据不存在则是NaN。\n12. **Embarked**：乘客上船的港口，S代表南安普敦，C代表 Cherbourg，Q代表 Queenstown。\n\n这些信息可以用来分析乘客的生存率与各种因素（如性别、舱位等级、年龄等）之间的关系。'}

# 智能体中的Token统计

In [7]:
from langchain.agents import AgentExecutor, Tool, create_openai_tools_agent
from langchain import hub
from langchain_zhipu import ChatZhipuAI

from langchain_core.callbacks import BaseCallbackHandler

class CountTokenCallbackHandler(BaseCallbackHandler):
	def on_llm_end(self, response, **kwargs):
		print("-"*20, "Token统计", "-"*20)
		print(response)
		print("-"*20, "--------", "-"*20)

tools = []
prompt = hub.pull("hwchase17/openai-tools-agent")
llm = ChatZhipuAI(model="glm-3-turbo", temperature=0.1, callbacks=[CountTokenCallbackHandler()])
agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

async for chunk in agent_executor.astream_events({"input": "你好"}, version="v1"):
    print(chunk)



> Entering new AgentExecutor chain...
{'event': 'on_chain_start', 'run_id': '70aa073c-cad2-4ca6-b78c-47d02a22153e', 'name': 'AgentExecutor', 'tags': [], 'metadata': {}, 'data': {'input': {'input': '你好'}}}
{'event': 'on_chain_start', 'name': 'RunnableSequence', 'run_id': '125f6b29-a71d-4777-ae2f-cc6027647970', 'tags': [], 'metadata': {}, 'data': {}}
{'event': 'on_chain_start', 'name': 'RunnableAssign<agent_scratchpad>', 'run_id': 'f7f62fd9-04dd-4312-9424-5d82d090357c', 'tags': ['seq:step:1'], 'metadata': {}, 'data': {}}
{'event': 'on_chain_stream', 'name': 'RunnableAssign<agent_scratchpad>', 'run_id': 'f7f62fd9-04dd-4312-9424-5d82d090357c', 'tags': ['seq:step:1'], 'metadata': {}, 'data': {'chunk': {'input': '你好', 'intermediate_steps': []}}}
{'event': 'on_chain_start', 'name': 'RunnableParallel<agent_scratchpad>', 'run_id': '5f173a3b-b9c5-4ace-8162-73605ebdbf7f', 'tags': [], 'metadata': {}, 'data': {}}
{'event': 'on_chain_start', 'name': 'RunnableLambda', 'run_id': '46692614-708c-4d49-

In [8]:
from langchain.agents import AgentExecutor, Tool, create_openai_tools_agent
from langchain.agents.agent import RunnableMultiActionAgent
from langchain import hub
from langchain_zhipu import ChatZhipuAI

from langchain_core.callbacks import BaseCallbackHandler

class CountTokenCallbackHandler(BaseCallbackHandler):
	def on_llm_end(self, response, **kwargs):
		print("-"*20, "Token统计", "-"*20)
		print(response)
		print("-"*20, "--------", "-"*20)

tools = []
prompt = hub.pull("hwchase17/openai-tools-agent")
llm = ChatZhipuAI(model="glm-3-turbo", temperature=0.1, callbacks=[CountTokenCallbackHandler()])
agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=RunnableMultiActionAgent(runnable =agent, stream_runnable=False), tools=tools, verbose=True)

async for chunk in agent_executor.astream_events({"input": "你好"}, version="v1"):
    print(chunk)



> Entering new AgentExecutor chain...
{'event': 'on_chain_start', 'run_id': '16145558-bbd5-41d0-a81d-5eafa081bece', 'name': 'AgentExecutor', 'tags': [], 'metadata': {}, 'data': {'input': {'input': '你好'}}}
{'event': 'on_chain_start', 'name': 'RunnableSequence', 'run_id': '9424c7a8-dfd2-476a-805a-0d2f6527f4bf', 'tags': [], 'metadata': {}, 'data': {}}
{'event': 'on_chain_start', 'name': 'RunnableAssign<agent_scratchpad>', 'run_id': '0aa170c3-ad83-487a-8e51-f11e129e9a4f', 'tags': ['seq:step:1'], 'metadata': {}, 'data': {}}
{'event': 'on_chain_stream', 'name': 'RunnableAssign<agent_scratchpad>', 'run_id': '0aa170c3-ad83-487a-8e51-f11e129e9a4f', 'tags': ['seq:step:1'], 'metadata': {}, 'data': {'chunk': {'input': '你好', 'intermediate_steps': []}}}
{'event': 'on_chain_start', 'name': 'RunnableParallel<agent_scratchpad>', 'run_id': 'd849659f-a959-48da-b08c-a30d5181d801', 'tags': [], 'metadata': {}, 'data': {}}
{'event': 'on_chain_start', 'name': 'RunnableLambda', 'run_id': '2344bde5-2b90-4929-